In [73]:
import pymysql 

conn = pymysql.connect(
    host="127.0.0.1",
    user="root",
    password="1234",
    database="olist",
)

## Read Query

In [21]:
select_sql = "SELECT * FROM reviews"
with conn.cursor() as cursor:
    cursor.execute(select_sql)
    result = cursor.fetchall()

In [22]:
result

((1, 1, 5),
 (2, 1, 4),
 (3, 1, 1),
 (4, 3, 2),
 (5, 3, 4),
 (6, 3, 3),
 (7, 4, 2),
 (8, 4, 3),
 (9, 4, 5),
 (10, 6, 5),
 (11, 6, 4),
 (12, 6, 1),
 (13, 6, 1))

## Write Query

In [20]:
insert_sql = "INSERT INTO reviews(item_id, rating) VALUES (6, 1)"
with conn.cursor() as cursor:
    cursor.execute(insert_sql)
    conn.commit()

## Bulk Write Query

In [52]:
insert_sql = "INSERT INTO customers(customer_id, customer_unique_id, customer_zip_code_prefix, customer_city, customer_state) VALUES (%s, %s, %s, %s, %s)"

In [74]:
import csv 

def _execute_insert(insert_sql, buffer):
    with conn.cursor() as cursor:
        cursor.executemany(insert_sql, buffer)
        conn.commit()
        
def bulk_insert(data_file_path, insert_sql, batchsize=1000):
    with open(data_file_path) as fr:
        reader = csv.reader(fr)
        buffer = []
        for i, row in enumerate(reader):
            row = [x if x else None for x in row]
            if i == 0:
                continue
            buffer.append(row)
            if len(buffer) == batchsize:
                _execute_insert(insert_sql, buffer)
                buffer = []
        if buffer:
            _execute_insert(insert_sql, buffer)

In [63]:
insert_order_items_sql = "INSERT INTO order_items(order_id, order_item_id, product_id, seller_id, shipping_limit_date, price, freight_value) VALUES(%s, %s, %s, %s, %s, %s, %s)"

In [44]:
bulk_insert("./data/olist/order_items.csv", insert_order_items_sql)

In [52]:
insert_reviews_sql = "INSERT INTO reviews(review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp) VALUES(%s, %s, %s, %s, %s, %s, %s)"

In [53]:
bulk_insert("./data/olist/reviews.csv", insert_reviews_sql)

In [64]:
insert_orders_sql = "INSERT INTO orders(order_id, customer_id, order_status, order_purchase_timestamp, order_approved_at, order_delivered_carrier_date, order_delivered_customer_date, order_estimated_delivery_date) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)"

In [75]:
bulk_insert("./data/olist/orders.csv", insert_orders_sql)

In [78]:
insert_products_sql = "INSERT INTO products(product_id, product_category_name, product_name_length, product_description_length, product_photos_qty, product_weight_g, product_length_cm, product_height_cm, product_width_cm) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)"

In [79]:
bulk_insert("./data/olist/products.csv", insert_products_sql)

In [80]:
insert_sellers_sql = "INSERT INTO sellers(seller_id, seller_zip_code_prefix, seller_city, seller_state) VALUES(%s, %s, %s, %s)"

In [81]:
bulk_insert("./data/olist/sellers.csv", insert_sellers_sql)

In [82]:
category_name_translation_sql = "INSERT INTO category_name_translation(product_category_name, product_category_name_english) VALUES(%s, %s)"

In [83]:
bulk_insert("./data/olist/category_name_translation.csv", category_name_translation_sql)

In [84]:
order_payments_sql = "INSERT INTO order_payments(order_id, payment_sequential, payment_type, payment_installments, payment_value) VALUES(%s, %s, %s, %s, %s)"

In [85]:
bulk_insert("./data/olist/order_payments.csv", order_payments_sql)

In [86]:
conn.close()